In [21]:
import os
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset
import random
import cv2
import os
import copy

In [22]:
CHARS = ['京', '沪', '津', '渝', '冀', '晋', '蒙', '辽', '吉', '黑',
         '苏', '浙', '皖', '闽', '赣', '鲁', '豫', '鄂', '湘', '粤',
         '桂', '琼', '川', '贵', '云', '藏', '陕', '甘', '青', '宁',
         '新',
         '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
         'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K',
         'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
         'W', 'X', 'Y', 'Z', 'I', 'O', '-'
         ]
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}

In [23]:
# 添加椒盐噪声
def SaltPepperNoise(src, percetage=0.3):
    h, w, c = src.shape
    SP_NoiseImg = src.copy()
    SP_NoiseNum = int(percetage*h*w)
    for i in range(SP_NoiseNum):
        randR = np.random.randint(0, h)
        randG = np.random.randint(0, w)
        randB = np.random.randint(0, 3)
        if random.random() > 0.5:
            SP_NoiseImg[randR, randG, randB] = 0
        else:
            SP_NoiseImg[randR, randG, randB] = 255
    return SP_NoiseImg

# 高斯噪声
def GaussianNoise(image, mean=0, sigma=1):
    image = image / 255.0
    noise = np.random.normal(mean, sigma, image.shape)
    G_Noiseimg = image + noise
    G_Noiseimg = np.clip(G_Noiseimg, a_max=255, a_min=0)
    G_Noiseimg = np.uint8(G_Noiseimg*255)
    return G_Noiseimg

# 旋转
def rotate(image, angle=90, center=None, scale=1.0):
    (h, w) = image.shape[:2]
    if center is None:
        center = (w / 2, h / 2)
    m = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, m, (w, h))
    return rotated

# 翻转
def flip(image):
    flipped_image = np.fliplr(image)
    return flipped_image

In [96]:
class LPRDataSet(Dataset):
    def __init__(self, img_dir, imgSize, lpr_max_len, PreprocFun=None, mode='test'):
        self.img_dir = img_dir
        self.img_paths = []
        if type(img_dir) == str:  # 一个路径
            for img_name in os.listdir(img_dir):
                img_name = os.path.join(img_dir, img_name)
                self.img_paths.append(img_name)
        else:
            print('error in img_dir, img_dir must be str or list')
        random.shuffle(self.img_paths)        # 打乱顺序
        self.img_size = imgSize
        self.lpr_max_len = lpr_max_len
        if PreprocFun is not None:
            self.PreprocFun = PreprocFun
        elif PreprocFun is None and mode == 'train':
            self.PreprocFun = self.train_transform
        elif PreprocFun is None and mode == 'test':
            self.PreprocFun = self.test_transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        filename = self.img_paths[index]
        image = cv2.imread(filename)
        height, width, _ = image.shape
        if width != self.img_size[0] or height != self.img_size[1]:
            image = cv2.resize(image, self.img_size)  # 缩放
        # image = image.astype('float32')
        # (height, width, channel) -> (channel, height, width) eg:(24,94,3)->(3,24,94) 
        # image = np.transpose(image, (2, 0, 1))
        image = self.PreprocFun(image)

        basename = os.path.basename(filename)
        imgname, suffix = os.path.splitext(basename)
        if int(imgname) < 9:
            imgname = '粤GDH078'
        else:
            imgname = '粤DSTU01'
        label = list()
        for c in imgname:
            label.append(CHARS_DICT[c])

        if len(label) > self.lpr_max_len:
            print(imgname)
            assert 0, "Error label ^~^!!!"

        return image, label, len(label)

    def test_transform(self, img):
        img = img.astype('float32')
        img -= 127.5
        img *= 0.0078125
        # (height, width, channel) -> (channel, height, width) eg:(24,94,3)->(3,24,94) 
        img = np.transpose(img, (2, 0, 1)) 
        return img

    def train_transform(self, img):
        img = img.astype('float32')
        p = random.uniform(0, 0.2)
        img = SaltPepperNoise(img, p)
        p = random.randint(0, 5)
        img = rotate(img, p)
        img -= 127.5
        img *= 0.0078125
        # (height, width, channel) -> (channel, height, width) eg:(24,94,3)->(3,24,94) 
        img = np.transpose(img, (2, 0, 1))
        return img

In [97]:
train_data = LPRDataSet('./data/demo/', (94, 24), 8, mode='train')
train_data.img_paths

In [103]:
img, label, length = train_data[0]
image = np.transpose(img, (1, 2, 0)) / 0.0078125 + 127.5
image = image.astype(np.uint8)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = Image.fromarray(image)
image.show()

In [82]:
test_data = LPRDataSet('./data/demo/', (94, 24), 8, mode='test')

In [84]:
img, label, length = test_data[0]
image = np.transpose(img, (1, 2, 0)) / 0.0078125 + 127.5
image = image.astype(np.uint8)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = Image.fromarray(image)
image.show()

In [45]:
def greedy_decode(preds, pred_char=False):
    last_chars_idx = len(CHARS) - 1

    # 贪婪解码 (greedy decode)
    pred_labels = []
    for i in range(preds.shape[0]):
        pred = preds[i, :, :]  # 第i张图片对应的结果，即维度为(66, 18)的二维数组
        pred_label = []
        for j in range(pred.shape[1]):  # 遍历每一列，找到每一列最大值的索引（index）
            pred_label.append(np.argmax(pred[:, j], axis=0))
        no_repeat_blank_label = []
        pre_c = -1
        for c in pred_label:  # 合并重复的索引值部分，删除空白标签，即为-1的值(dropout repeate label and blank label)
            if (pre_c == c) or (c == last_chars_idx):
                if c == last_chars_idx:
                    pre_c = c
                continue
            no_repeat_blank_label.append(c)
            pre_c = c
        pred_labels.append(no_repeat_blank_label)

    # 解码成字符串
    if pred_char:
        labels = []
        for label in pred_labels:
            lb = ""
            for i in label:
                lb += CHARS[i]
            labels.append(lb)
        return pred_labels, labels
    else:
        return pred_labels

In [13]:
import torch
import torch.nn as nn

In [14]:
class small_basic_block(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(small_basic_block, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(ch_in, ch_out // 4, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out, kernel_size=1),
        )
    def forward(self, x):
        return self.block(x)


class LPRNet(nn.Module):
    def __init__(self, lpr_max_len, class_num, dropout_rate):
        super(LPRNet, self).__init__()
        self.lpr_max_len = lpr_max_len
        self.class_num = class_num
        self.backbone = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1), # 0
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),  # 2
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1)),
            small_basic_block(ch_in=64, ch_out=128),    # *** 4 ***
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),  # 6
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2)),
            small_basic_block(ch_in=64, ch_out=256),   # 8
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),  # 10
            small_basic_block(ch_in=256, ch_out=256),   # *** 11 ***
            nn.BatchNorm2d(num_features=256),   # 12
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(4, 1, 2)),  # 14
            nn.Dropout(dropout_rate),
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=(1, 4), stride=1),  # 16
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),  # 18
            nn.Dropout(dropout_rate),
            nn.Conv2d(in_channels=256, out_channels=class_num, kernel_size=(13, 1), stride=1), # 20
            nn.BatchNorm2d(num_features=class_num),
            nn.ReLU(),  # *** 22 ***
        )
        self.container = nn.Sequential(
            nn.Conv2d(in_channels=448+self.class_num, out_channels=self.class_num, kernel_size=(1, 1), stride=(1, 1)),
            # nn.BatchNorm2d(num_features=self.class_num),
            # nn.ReLU(),
            # nn.Conv2d(in_channels=self.class_num, out_channels=self.lpr_max_len+1, kernel_size=3, stride=2),
            # nn.ReLU(),
        )

    def forward(self, x):
        keep_features = list()
        for i, layer in enumerate(self.backbone.children()):
            x = layer(x)
            if i in [2, 6, 13, 22]: # [2, 4, 8, 11, 22]
                keep_features.append(x)

        global_context = list()
        for i, f in enumerate(keep_features):
            if i in [0, 1]:
                f = nn.AvgPool2d(kernel_size=5, stride=5)(f)
            if i in [2]:
                f = nn.AvgPool2d(kernel_size=(4, 10), stride=(4, 2))(f)
            f_pow = torch.pow(f, 2)
            f_mean = torch.mean(f_pow)
            f = torch.div(f, f_mean)
            global_context.append(f)

        x = torch.cat(global_context, 1)
        x = self.container(x)
        logits = torch.mean(x, dim=2)

        return logits

In [15]:
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

In [51]:
train_data = LPRDataSet('./data/demo/', (94, 24), 8, mode='train')
test_data = LPRDataSet('./data/demo/', (94, 24), 8, mode='test')

In [38]:
model = LPRNet(lpr_max_len=8, class_num=len(CHARS), dropout_rate=0.5)
dic = torch.load('./weights/Final_LPRNet_model.pth', map_location=torch.device("cpu"))
model.load_state_dict(dic)

<All keys matched successfully>

In [106]:
epoch = 1

# 损失函数 reduction: 'none' | 'mean' | 'sum'
ctc_loss = nn.CTCLoss(blank=len(CHARS)-1, reduction='mean')

# 优化器
optimizer = optim.Adam(
    model.parameters(),
    lr=0.001
)

# cpu训练还是GPU训练
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('using ', device)
model = model.to(device)

# tensorboard
log_folder = "./logs/demo"
if not os.path.exists(log_folder):
    os.mkdir(log_folder)
writer = SummaryWriter(log_dir=log_folder)

# 模型保存路径
save_folder = './weights'
if not os.path.exists(save_folder):
    os.mkdir(save_folder)

using  cpu


In [107]:
batch_size = 20

def collate_fn(batch):
    imgs = []
    labels = []
    lengths = []
    for sample in batch:
        img, label, length = sample
        imgs.append(torch.from_numpy(img))
        # imgs.append(img)
        labels.extend(label)
        lengths.append(length)
    labels = np.asarray(labels).flatten().astype(np.int32)
    return (torch.stack(imgs, 0), torch.from_numpy(labels), lengths)


# 利用DataLoader加载数据集
train_loader = DataLoader(dataset=train_data, batch_size=batch_size,
                          shuffle=True, drop_last=True, collate_fn=collate_fn)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, collate_fn=collate_fn,
                         shuffle=True, drop_last=True)

In [108]:
# https://zhuanlan.zhihu.com/p/67415439
def sparse_tuple_for_ctc(T_length=18, lengths=8):
    # T_length >= 2*lengths+1, lengths <= lpr_max_len
    input_lengths = []
    target_lengths = []

    for ch in lengths:
        input_lengths.append(T_length)
        target_lengths.append(ch)

    return tuple(input_lengths), tuple(target_lengths)

In [110]:
# 训练和测试
verbose_step = 1
total_train_step = 0  # 记录训练次数
total_test_step = 0   # 记录测试次数
acc = 0.55
for i in range(epoch):
    print("-----第{}轮训练开始-----".format(i+1))

    # 训练
    model.train()  # 开启训练模式
    for images, labels, lengths in train_loader:
        # get ctc parameters
        input_lengths, target_lengths = sparse_tuple_for_ctc(lengths=lengths)
        # forward
        images = images.to(device)
        logits = model(images)
        log_probs = logits.permute(2, 0, 1) # for ctc loss: T x N x C
        log_probs = log_probs.log_softmax(2).requires_grad_()
        # backprop
        optimizer.zero_grad()  # 优化器中的参数梯度归零
        loss = ctc_loss(log_probs, labels, input_lengths=input_lengths, target_lengths=target_lengths)
        if loss.item() == np.inf:  # 如果梯度爆炸
            continue
        loss.backward()        # 反向传播，计算梯度
        optimizer.step()       # 优化网络
        # 记录loss
        if total_train_step % verbose_step == 0:  # 每verbose_step个batch，显示信息
            print("训练次数：{}，loss：{}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
            writer.add_images("Epoch".format(epoch), images, total_train_step)
        total_train_step += 1

    # 测试
    model.eval()  # 开启测试模式
    total_test_loss = 0
    total_acc_num = 0
    Tp, Tn = 0.0, 0.0
    with torch.no_grad():  # 关闭梯度计算
        for images, labels, lengths in test_loader:
            # labels: 1D -> 2D
            targets = []
            start = 0
            for length in lengths:
                label = labels[start:start+length]
                targets.append(label.tolist())
                start += length
            # forward
            images = images.to(device)
            prebs = model(images)
            prebs = prebs.cpu().detach().numpy()
            # greedy decode
            preb_labels = greedy_decode(prebs)
            # calculate
            for i, label in enumerate(preb_labels):
                if len(label) != len(targets[i]):  # 长度不一致
                    Tn += 1
                    continue
                if targets[i] == label:
                    Tp += 1
                else:
                    Tn += 1
        total_test_acc = float(Tp) / float(Tp + Tn)
        print("整体测试集上的acc: {}".format(total_test_acc))
        writer.add_scalar("test_acc", total_test_acc, total_test_step)
        total_test_step += 1

    # 保存模型参数
    if total_test_acc >= acc:  # 保存最好的
        acc = total_test_acc
        model_path = save_folder + "/demo_{:.2f}.pth".format(acc)
        torch.save(model.state_dict(), model_path)
        print("模型已保存")

writer.close()

-----第1轮训练开始-----
训练次数：0，loss：17.994121551513672
整体测试集上的acc: 0.0


### 查看acc/loss曲线、训练的照片

In [ ]:
!tensorboard --logdir=logs

### 测试

In [111]:
# 测试
Tp, Tn = 0.0, 0.0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.eval()  # 开启测试模式
with torch.no_grad():  # 关闭梯度计算
    for images, labels, lengths in test_loader:
        # labels: 1D -> 2D
        targets = []
        start = 0
        for length in lengths:
            label = labels[start:start+length]
            targets.append(label.tolist())
            start += length
        # forward
        images = images.to(device)
        prebs = model(images)
        prebs = prebs.cpu().detach().numpy()
        # greedy decode
        preb_labels = greedy_decode(prebs)
        # calculate
        for i, label in enumerate(preb_labels):
            if len(label) != len(targets[i]):  # 长度不一致
                Tn += 1
                continue
            if targets[i] == label:
                Tp += 1
            else:
                Tn += 1
    total_test_acc = float(Tp) / float(Tp + Tn)
    print("整体测试集上的acc: {}".format(total_test_acc))

整体测试集上的acc: 0.0
